## Requisitos

Si no lo tienes instalar, revise la documentacion para mas informacion [Docu](https://pypi.org/project/coboljsonifier/)
> pip install coboljsonifier

In [2]:
import json
from coboljsonifier.copybookextractor import CopybookExtractor
from coboljsonifier.parser import Parser
from coboljsonifier.config.parser_type_enum import ParseType
import pandas as pd
import ast

pd.options.display.max_rows = 4000

In [16]:
bookfname = 'bgdtacu.cob'
interface = 'BGDTACU-20230727.DAT'
rows = 2

### Building a Parser

Use book structure to build a parser (**FLAT_ASCII** / **BINARY_EBCDIC**)

In [17]:
dict_structure = CopybookExtractor(bookfname).dict_book_structure

df = pd.DataFrame([key for key in dict_structure.keys()], columns=['Name'])
df['type'] = [value['type'] for value in dict_structure.values()]
df['format'] = [value['format'] for value in dict_structure.values()]
df['size'] = [value['size'] for value in dict_structure.values()]
df['decimals'] = [value['decimals'] for value in dict_structure.values()]

In [5]:
def parse_ascii(dict_structure,interface,df):
    parser = Parser(dict_structure, ParseType.FLAT_ASCII).build()

    with open(interface) as archivo:
        i = 1
        for linea in archivo:
            
            data= linea
            parser.parse(data)
            dictvalue = parser.value
            pretty = json.dumps(dictvalue, indent=4,default=str)
            val_convert = ast.literal_eval(pretty.replace("null", "None"))

            col = 'col' + str(i)
            df[col] = [value for value in val_convert.values()]

            if i==rows:
                break
            
            i = i + 1

In [18]:
def parse_bin(dict_structure,interface, df):
    parser = Parser(dict_structure, ParseType.BINARY_EBCDIC).build()
    size = parser.size
    print("// Registry calculated lenght:", size)

    with open(interface, 'rb') as f2:
        for x in range(rows):

            # EBCDIC
            data = f2.read(size)
            if not data:
                break
           
            if data[0:2] == b'\xF0\xF0':    # ====> El nuestro es este formato
                parser.parse(data)
                dictvalue = parser.value

                pretty = json.dumps(dictvalue, indent=4,default=str)
                val_convert = ast.literal_eval(pretty.replace("null", "None"))
                col = 'col' + str(x)
                df[col] = [value for value in val_convert.values()]

            else:
                print(f'// Registry type {data[0:2]} not processed')


In [6]:
parse_ascii(dict_structure,interface, df)

In [19]:
parse_bin(dict_structure,interface, df)

// Registry calculated lenght: 243


In [20]:
#TODO: 2 Unir la implementacion para que todo quede en un solo nootbook
df

,Name,type,format,size,decimals,col0,col1,col2
0,ENTIDAD,ALPHANUMERIC,X(04),4,0,0072,0072,0072
1,CENTRO-ALTA,ALPHANUMERIC,X(04),4,0,0000,0000,0000
2,CUENTA,ALPHANUMERIC,X(12),12,0,005000021937,005000021937,005000021937
3,DIVISA,ALPHANUMERIC,X(3),3,0,ARS,ARS,ARS
4,SECUENCIA,NUMERIC_COMP3,S9(5),3,0,2037,2039,2040
5,TIPO-ACUERDO,ALPHANUMERIC,X(1),1,0,E,E,E
6,ESTADO,ALPHANUMERIC,X(1),1,0,V,V,A
7,LIMITE,NUMERIC_COMP3,S9(13)V9(2),8,2,375000000.00,375000000.00,1210000000.00
8,LIMITE-RENOV,NUMERIC_COMP3,S9(13)V9(2),8,2,375000000.00,375000000.00,1210000000.00
9,PERIOD-LIQ,ALPHANUMERIC,X(1),1,0,V,V,V


## Posibles errores

En **Extracting copybook structure** le da error los  |    PIC X.   => Cambiarlos por =>   PIC X(1).  en el copybook